# Lab 5 - general solutions of univariate models

OK, this week we'll see how we can use SageMath to solve for general solutions, ie our variable of interest as a function of time. 

## Discrete time

To solve recursion equations in SageMath we need to explicitly use Maxima (one of the main engines behind SageMath)

In [1]:
maxima.load('solve_rec'); #load this maxima function

Now solve the exponential growth model

In [4]:
sol = maxima('solve_rec(n[t+1]=R*n[t], n[t], n[0]=n0)') #solve the recurrence equation for exponential growth
show(sol)

n[t]=R^t*n0

This also works for affine models

In [5]:
sol = maxima('solve_rec(n[t+1]=R*n[t]+m, n[t], n[0]=n0)') #solve the recurrence equation for exponential growth with migration
show(sol)

n[t]=R^t*n0+(R^t*m)/(R-1)-m/(R-1)

And for the model of haploid selection

In [6]:
sol = maxima('solve_rec(p[t+1]=(p[t]*WA)/(p[t]*WA+(1-p[t])*Wa), p[t], p[0]=p0)') #solve the recurrence equation for haploid selection
show(sol.factor())

p[t]=-((-WA/(Wa-WA))^t*p0)/((-Wa/(Wa-WA))^t*p0-(-WA/(Wa-WA))^t*p0-(-Wa/(Wa-WA))^t)

But let's skip to continuous time, where we can solve more equations

## Continuous time

In lab 2, when introducing differential equations, we already saw how we can use ```desolve``` to get the general solutions for exponential growth, logistic growth, and haploid selection

In [3]:
# exponential growth

r,t,n0 = var('r, t, n0') #define the parameters
n = function('n')(t) #define a general function

f = desolve(
    diff(n(t),t) == r * n(t), #the differential equation we want to solve
    n(t), #the function we want to solve for
    ivar = t, #the independent variable we want the function to be of
    ics = [0, n0] #the initial conditions (t0, n0)
)

show(f)

n0*e^(r*t)

In [8]:
# logistic growth

r,k,t,n0 = var('r, k, t, n0') #define the parameters
n = function('n')(t) #define a general function

f = desolve(
    diff(n(t),t) == r * n(t) * (1 - n(t) / k), #the differential equation we want to solve
    n(t), #the function we want to solve for
    ivar = t, #the independent variable we want the function to be of
    ics = [0, n0] #the initial conditions (t0, n0)
)

#next a little trick to replace our function n with a variable z
z = SR.var('z')
fz = f.substitute({n:z})

#and then we can solve for z=n explicitly with sympy
fz.solve(z, algorithm='sympy')

#note the set notation A \ B means for all A that are not in B (so in this case we're ignoring the scenario where n(t)=0)

Complement(FiniteSet(k*n0*exp(r*t)/(k + n0*exp(r*t) - n0)), FiniteSet(0))

In [12]:
# haploid selection

s,t,p0 = var('s, t, p0') #define the parameters
p = function('p')(t) #define a general function

f = desolve(
    diff(p(t),t) == s * p(t) * (1 - p(t)), #the differential equation we want to solve
    p(t), #the function we want to solve for
    ivar = t, #the independent variable we want the function to be of
    ics = [0, p0] #the initial conditions (t0, p0)
)

#next a little trick to replace our function p with a variable z
z = SR.var('z')
fz = f.substitute({p:z})

#and then we can solve for z=p explicitly with sympy
fz.solve(z, algorithm='sympy')

Complement(FiniteSet(p0*exp(s*t)/(p0*exp(s*t) - p0 + 1)), FiniteSet(0))

And in the lectures we saw how to solve these using seperation of variables.

Now let's move on to some differential equations that require other methods to solve (see Box 6.2 in the text for how to do this by hand).

**Q1** Sink populations are defined as populations that are maintained by migration from another source population. Here we consider a sink population with a negative intrinsic growth rate that receives immigrants at rate $m$ per year, which is decreasing over time due to habitat deterioration in migration corridors, $m=m-m_0-\delta t$. A differential equation describing this situation is 

$\frac{\mathrm{d}n}{\mathrm{d}t} = r n + (m_0 - \delta t)$.

(a) [1 point] Use ```desolve``` to get the general solution, $n(t)$

(b) [1 point] Plot this solution from $t=0$ to $t=80$ using $r=-0.01$, $m_0=0.5$, $\delta=0.02$, and $n(0)=40$

(c) [1 point] Plot the dynamics from just $t=0$ to $t=10$: why, biologicall, do we see these dynamics?

(d) [1 point] When do you expect the population to go extinct? [hint: sagemath's ```solve``` has some trouble with this, so try ```find_root``` -- type ```find_root?``` to see how this function works]

**Q2** (a) [1 point] Solve the haploid model of selection in continuous time, $\frac{\mathrm{d}p}{\mathrm{d}t}=sp(1-p)$, under the assumption that the selection coefficient varies sinusoidally with time, $s = s_0 + \sigma sin(\theta t)$, as might be the case in a seasonal environment.

(b) [1 point] Plot this for $s_0=0.01$, $\sigma=0.1$, $\theta=1$, $p_0=0.5$ from $t=0$ to $t=100$

(c) [1 point] What happens as time goes to infinity when $s_0>0$? And $s_0<0$?

**Q3** By decreasing the density of resources available, habitat degredation could act to reduce the reproductive potential of a species or to increase the amount of territory needed to sustain an individual.

(a) [1 point] Solve the logistic growth model in continuous time, $\frac{\mathrm{d}n}{\mathrm{d}t} = r n (1 - n/K)$, when $r(t) = r_0 (1 - \delta t)$ but $K(t)=K_0$ is constant.

(b) [1 point] Now solve the logistic growth model when $r(t)=r_0$ is constant but $K(t)=K_0/(1 + \delta t)$.

(c) [1 point] Compare these by plotting them for $n(0)=500$, $r_0=1$, $K_0=1000$, and $\delta=0.1$. Which is expected to go extinct first?